In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/Users/keemsunguk/Projects/EssayGrader/')

In [4]:
import json
import re
import spacy
from spacy import displacy

In [5]:
from egrader.preprocess import Preprocess
from egrader.db_util import DBUtil

In [6]:
from pycorenlp import StanfordCoreNLP

In [7]:
prep = Preprocess(None)

Loaded model 'en_core_web_lg'


In [8]:
db_util = DBUtil(local_db=True)
db_util.describe_db()

==== LOCAL ====
['essay']
Total Essay: 30498
Total SAT: 3310
Total TOEFL: 4614
Total GRE: 6688
==== REMOTE ====
['essays']
Total Essay: 15028
Total SAT: 0
Total TOEFL: 4613
Total GRE: 6662


In [7]:
txt = db_util.remote_ec.find_one({})['essay']
topic = db_util.remote_ec.find_one({})['topic']

In [8]:
db_util.local_ec.find_one({}).keys()

dict_keys(['_id', 'recno', 'user_id', 'type', 'essay', 'time_stamp', 'created', 'reviewed', 'edited', 'topic', 'rate2', 'rate', 'editor', 'wcount', 'sentences'])

In [9]:
db_util.remote_ec.find_one({}).keys()

dict_keys(['_id', 'essay', 'time_stamp', 'user_id', 'wcount', 'topic', 'edited', 'type', 'created', 'recno', 'editor', 'rate2', 'rate', 'reviewed', 'sentences'])

In [10]:
topic = prep.clean_html(topic)
topic_doc = prep.make_doc_obj()
txt = prep.clean_html(txt)
txt_doc = prep.make_doc_obj()

In [11]:
displacy.render(txt_doc, style="ent", jupyter=True)

In [12]:
displacy.render(topic_doc, style='ent', jupyter=True)

In [20]:
import stanza
stanza.download('en')   # This downloads the English models for the neural pipeline

2020-03-29 13:59:20 INFO: Downloading default packages for language: en (English)...
2020-03-29 13:59:21 INFO: File exists: /Users/keemsunguk/stanza_resources/en/default.zip.
2020-03-29 13:59:25 INFO: Finished downloading models and saved to /Users/keemsunguk/stanza_resources.


In [9]:
models_dir = '/Users/keemsunguk/Projects/stanfordnlp'
#nlp = stanfordnlp.Pipeline(models_dir=models_dir, lang='en', use_gpu=False) # This sets up a default neural pipeline in English
#essay_doc = nlp(txt)
#essay_doc.sentences[0].print_dependencies()

In [21]:
dir = '/Users/keemsunguk/stanza_resources'
nlp2 = stanza.Pipeline(dir=dir, lang='en', use_gpu=False)
edoc = nlp2('The letter from a committee contains an assertion that there should be restrictions on landscaping and housepainting to raise the property values .')

2020-03-29 13:59:30 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| ner       | ontonotes |

2020-03-29 13:59:30 INFO: Use device: cpu
2020-03-29 13:59:30 INFO: Loading: tokenize
2020-03-29 13:59:30 INFO: Loading: pos
2020-03-29 13:59:31 INFO: Loading: lemma
2020-03-29 13:59:31 INFO: Loading: depparse
2020-03-29 13:59:32 INFO: Loading: ner
2020-03-29 13:59:33 INFO: Done loading processors!


In [11]:
edoc.to_dict()[0][0]

{'id': '1',
 'text': 'The',
 'lemma': 'the',
 'upos': 'DET',
 'xpos': 'DT',
 'feats': 'Definite=Def|PronType=Art',
 'head': 2,
 'deprel': 'det',
 'misc': 'start_char=0|end_char=3'}

In [68]:
db_util.local_ec.find_one({})['sentences'][0]

{'text': 'The letter from a committee contains an assertion that there should be restrictions on landscaping and housepainting to raise the property values .',
 'dependencies': [{'governer': {'index': '1',
    'text': 'The',
    'lemma': 'the',
    'upos': 'DET',
    'xpos': 'DT',
    'feats': 'Definite=Def|PronType=Art',
    'governor': 2,
    'dependency_relation': 'det'},
   'edge': 'det',
   'node': {'index': '2',
    'text': 'letter',
    'lemma': 'letter',
    'upos': 'NOUN',
    'xpos': 'NN',
    'feats': 'Number=Sing',
    'governor': 6,
    'dependency_relation': 'nsubj'}},
  {'governer': {'index': '2',
    'text': 'letter',
    'lemma': 'letter',
    'upos': 'NOUN',
    'xpos': 'NN',
    'feats': 'Number=Sing',
    'governor': 6,
    'dependency_relation': 'nsubj'},
   'edge': 'nsubj',
   'node': {'index': '6',
    'text': 'contains',
    'lemma': 'contain',
    'upos': 'VERB',
    'xpos': 'VBZ',
    'feats': 'Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin',
    'gover

In [46]:
essay_doc.write_conll_to_file('/Users/keemsunguk/Projects/data/corenlp_text.txt')

In [15]:
def jsonify_corenlp_sentence(sent: 'list of dependencies') -> dict:
    dependency_list = []
    for dep in sent.dependencies:
        dependency_list.append({
            'governer': {
                'index': dep[2].index,
                'text': dep[2].text,
                'lemma': dep[2].lemma,
                'upos': dep[2].upos,
                'xpos': dep[2].xpos,
                'feats': dep[2].feats,
                'governor': dep[2].governor,
                'dependency_relation': dep[2].dependency_relation
            },
            'edge': dep[1],
            'node': {
                'index': dep[0].index,
                'text': dep[0].text,
                'lemma': dep[0].lemma,
                'upos': dep[0].upos,
                'xpos': dep[0].xpos,
                'feats': dep[0].feats,
                'governor': dep[0].governor,
                'dependency_relation': dep[0].dependency_relation                
            }
        })
    return dependency_list

In [82]:
essay_doc.sentences[0].words

[<Word index=1;text=The;lemma=the;upos=DET;xpos=DT;feats=Definite=Def|PronType=Art;governor=2;dependency_relation=det>,
 <Word index=2;text=letter;lemma=letter;upos=NOUN;xpos=NN;feats=Number=Sing;governor=6;dependency_relation=nsubj>,
 <Word index=3;text=from;lemma=from;upos=ADP;xpos=IN;feats=_;governor=5;dependency_relation=case>,
 <Word index=4;text=a;lemma=a;upos=DET;xpos=DT;feats=Definite=Ind|PronType=Art;governor=5;dependency_relation=det>,
 <Word index=5;text=committee;lemma=committee;upos=NOUN;xpos=NN;feats=Number=Sing;governor=2;dependency_relation=nmod>,
 <Word index=6;text=contains;lemma=contain;upos=VERB;xpos=VBZ;feats=Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin;governor=0;dependency_relation=root>,
 <Word index=7;text=an;lemma=a;upos=DET;xpos=DT;feats=Definite=Ind|PronType=Art;governor=8;dependency_relation=det>,
 <Word index=8;text=assertion;lemma=assertion;upos=NOUN;xpos=NN;feats=Number=Sing;governor=6;dependency_relation=obj>,
 <Word index=9;text=that;lemma=tha

In [85]:
sentences = []
for sent in essay_doc.sentences:
    sentence = dict()
    sentence['text'] = ' '.join([w.text for w in sent.words])
    sentence['dependencies'] = jsonify_corenlp_sentence(sent)
    sentences.append(sentence)

# now parse all essays

In [ ]:
# Remote
for e in db_util.remote_ec.find({}):
    essay = e['essay']
    txt = prep.clean_html(essay)
    essay_doc = nlp(txt)
    sentences = []
    for sent in essay_doc.sentences:
        sentence = dict()
        sentence['text'] = ' '.join([w.text for w in sent.words])
        sentence['dependencies'] = jsonify_corenlp_sentence(sent)
        sentences.append(sentence)
    db_util.remote_ec.update_one({'_id':e['_id']}, {'$set':{'sentences': sentences}}, upsert=False)

In [ ]:
# Local
for e in db_util.local_ec.find({}):
    essay = e['essay']
    txt = prep.clean_html(essay)
    essay_doc = nlp(txt)
    sentences = []
    for sent in essay_doc.sentences:
        sentence = dict()
        sentence['text'] = ' '.join([w.text for w in sent.words])
        sentence['dependencies'] = jsonify_corenlp_sentence(sent)
        sentences.append(sentence)
    db_util.local_ec.update_one({'_id':e['_id']}, {'$set':{'sentences': sentences}}, upsert=False)

../aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.
../aten/src/ATe

In [ ]:
db_util.local_ec.find_one({})

In [16]:
have_sentences = 0
redo_list = []
for r in db_util.local_ec.find({}):
    try:
        if len(r['sentences']) > 0:
            have_sentences += 1
        else:
            print('Have sentence but 0 size', r['recno'])
    except:
        redo_list.append(r['recno'])
print(have_sentences)

4270


In [17]:
print(len(redo_list))

27976


In [19]:
for eid in redo_list:
    e = db_util.local_ec.find_one({'recno': eid})
    essay = e['essay']
    if e['type'].lower() == 'gre':
        txt = prep.clean_html(essay)
        essay_doc = nlp(txt)
        sentences = []
        for sent in essay_doc.sentences:
            sentence = dict()
            sentence['text'] = ' '.join([w.text for w in sent.words])
            sentence['dependencies'] = jsonify_corenlp_sentence(sent)
            sentences.append(sentence)
        db_util.local_ec.update_one({'recno':eid}, {'$set':{'sentences': sentences}}, upsert=False)
        print(eid)

../aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


5062


../aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


5064


../aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


...

33137


../aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


33138


../aten/src/ATen/native/LegacyDefinitions.cpp:19: UserWarning: masked_fill_ received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead.


33173


In [22]:
db_util.local_ec.find_one({'recno': 5062})

{'_id': ObjectId('5cd245d7f27f8d12134f60d5'),
 'recno': 5062,
 'user_id': 'beeedan@yahoo.co.kr',
 'type': 'TOEFL',
 'essay': "In most big cities, we hear controversies over the priority in traffic policy: Which is more necessary? more roads? or more public transportation? In mayoral elections of major cities, this is a big issue as well.\r\n\r\nThe answer lies in which you-policymakers- think is more important. If you put more stress on the rights of the peole who own their own cars, commute by their cars, then the answer may be more roads. But if you consider more of the rights of ordinary peole commuting by public transportations, then the answer is, of course, more money on improving public transportation.\r\n\r\nNot only studies but also public experience show that, newly built roads which were originally built to address heavy traffic jams are soon filled with evev more cars, and the traffic gets heavier, sometimes worse, again. That is why many experts on  traffic issues argue th

In [52]:
selected_essay = [e for e in db_util.local_ec.find({'$and': [{'type': 'toefl'.upper()}, {'rate': {'$gte': 0}}]}, limit=2) if len(e['essay']) > 100]
result_dict = dict()
for e in selected_essay:
    result_dict[str(e['_id'])] = e
    result_dict[str(e['_id'])].pop('_id', None)
with open('/Users/keemsunguk/Projects/data/test.json', 'w') as wf: 
    json.dump(result_dict, wf, indent=4)

In [56]:
selected_essay = [e for e in db_util.local_ec.find({'$and': [{'type': 'toefl'.upper()}, {'rate': {'$gte': 0}}]}, limit=2) if len(e['essay']) > 100]
with open('/Users/keemsunguk/Projects/data/test.json', 'w') as wf:
    for e in selected_essay:
        e['_id'] = str(e['_id'])
    json.dump(selected_essay, wf, indent=4)

In [50]:
result_dict

{'5cd245d5f27f8d12134f505e': {'recno': 458,
  'user_id': 'jieun_l@hotmail.com',
  'type': 'TOEFL',
  'essay': 'Having a new large factory can have both advantages and disadvantages in the community. As far as I am concerned, I oppose to buid a ne factory in my neighborhood. This is becuase there is more disadvantages of having a factory to local residents than benefits.\r\n\r\nA new factory could be beneficial in a place like ghetto, where many beggars and unemploys live. People in here can get a job and earn enough money so that they could eat and support their family. These days, many people are out of work and depend upon the welfare system. Therefore, a new factory could be a kind of hope for these unemployed people. Also, once a ew big business starts up, other small buisinesses tend to follow. From this community will have a chance to grow and develop.\r\n\r\nOn the other hand, people should consider about the environmental problem factory will bring. Tody, pollution is the bigge

In [96]:
db_util.remote_ec.update_one({'recno':recno}, {'$set':{'sentences': None}}, upsert=False)

In [18]:
for r in db_util.remote_ec.find({'$and':[{'type': 'TOEFL'}, {'recno':{'$gt': 10179}}]}, {'recno':1, 'essay':1}):
    recno = r['recno']
    try:
        one_doc = nlp2(r['essay'])
        db_util.remote_ec.update_one({'recno':recno}, {'$set':{'sentences': one_doc.to_dict()}}, upsert=False)
        print(recno)
    except:
        print('=======================', recno)
    #break

10183
10184
10192
10194
10196
10198
10199
10205
10206
10208
10211
10212
10213
10214
10215
10217
10219
10282
10223
10225
10228
10230
10232
10233
10234
10240
10250
10256
10257
10261
10266
10279
10288
10292
10302
10305
10307
10310
10311
10314
10315
10330
10345
10347
10350
10358
10363
10364
10367
10370
10371
10375
10376
10379
10384
10395
10396
10397
10403
10409
10410
10411
10412
10413
10414
10415
10420
10423
10424
10427
10430
10439
10440
10441
10442
10443
10445
10447
10449
10451
10454
10459
10462
10463
10465
10470
10480
10481
10483
10486
10499
10502
10504
10505
10511
10513
10514
10520
10521
10522
10523
10524
10526
10528
10530
10531
10532
10533
10534
10536
10537
10538
10539
10541
10543
10545
10546
10560
10561
10562
10563
10564
10566
10567
10568
10569
10575
10576
10577
10579
10580
10583
10584
10591
10593
10595
10596
10597
10599
10600
10604
10616
10619
10620
10623
10624
10625
10629
10631
10632
10636
10637
10638
10643
10648
10649
10650
10651
10664
10665
10667
10669
10673
10675
10679
10681
1068

In [8]:
gre_list = [r['recno'] for r in db_util.remote_ec.find({'type': 'GRE'})]

In [13]:
# Remove dependency data from remote
for r in gre_list[1000:]:
    try:
        db_util.remote_ec.update_one({'recno':r}, {'$set':{'sentences': None}}, upsert=False)
        #print(r)
    except:
        print('=======================', r)

In [9]:
no_type_list = [r['recno'] for r in db_util.remote_ec.find({'type': ''})]
print(len(no_type_list))

547


In [14]:
# Remove dependency data from remote
for r in no_type_list:
    try:
        db_util.remote_ec.update_one({'recno':r}, {'$set':{'sentences': None}}, upsert=False)
        #print(r)
    except:
        print('=======================', r)

In [25]:
tmp = db_util.local_ec.find_one({'recno': 458})

In [23]:
cnlp = StanfordCoreNLP('http://localhost:9000')

In [26]:
# Test
output = cnlp.annotate(tmp['essay'], properties={
  'annotators': 'tokenize,ssplit,pos,parse,coref',
  'outputFormat': 'json'
  })

In [59]:
for s in output['sentences'][:1]:
    for k, v in s.items():
        if k == 'parse':
            print(v)
        elif k == 'enhancedPlusPlusDependencies':
            print(v)

(ROOT
  (S
    (S
      (VP (VBG Having)
        (NP (DT a) (JJ new) (JJ large) (NN factory))))
    (VP (MD can)
      (VP (VB have)
        (NP
          (NP (DT both) (NNS advantages)
            (CC and)
            (NNS disadvantages))
          (PP (IN in)
            (NP (DT the) (NN community))))))
    (. .)))
[{'dep': 'ROOT', 'governor': 0, 'governorGloss': 'ROOT', 'dependent': 7, 'dependentGloss': 'have'}, {'dep': 'csubj', 'governor': 7, 'governorGloss': 'have', 'dependent': 1, 'dependentGloss': 'Having'}, {'dep': 'det', 'governor': 5, 'governorGloss': 'factory', 'dependent': 2, 'dependentGloss': 'a'}, {'dep': 'amod', 'governor': 5, 'governorGloss': 'factory', 'dependent': 3, 'dependentGloss': 'new'}, {'dep': 'amod', 'governor': 5, 'governorGloss': 'factory', 'dependent': 4, 'dependentGloss': 'large'}, {'dep': 'dobj', 'governor': 1, 'governorGloss': 'Having', 'dependent': 5, 'dependentGloss': 'factory'}, {'dep': 'aux', 'governor': 7, 'governorGloss': 'have', 'dependent': 6, 'd

In [35]:
selected_essay = [e for e in db_util.local_ec.find({'$and': [{'type': 'TOEFL'}, {'rate': {'$gte': 0}}, {'recno': {'$gt':32631}}]}, {'recno':1, 'essay':1})]
len(selected_essay)

173

In [36]:
# Remove dependency data from remote
for e in selected_essay:
    try:
        output = cnlp.annotate(e['essay'], properties={
              'annotators': 'tokenize,ssplit,pos,parse,coref',
              'outputFormat': 'json'
          })
        data = [{'parse': o['parse'], 'enhancedpp': o['enhancedPlusPlusDependencies']} for o in output['sentences']]
        db_util.remote_ec.update_one({'recno':e['recno']}, {'$set':{'constituency': data}}, upsert=False)
        db_util.local_ec.update_one({'recno':e['recno']}, {'$set':{'sentences': data}}, upsert=False)
        #print(e['recno'])
    except:
        print('=======================', e['recno'])

In [33]:
db_util.remote_ec.delete_many({'type': 'BEG'})

In [34]:
db_util.remote_ec.distinct('type')

['TOEFL']

In [38]:
db_util.remote_ec.find_one({})

{'_id': ObjectId('5cd245d5f27f8d12134f505e'),
 'essay': 'Having a new large factory can have both advantages and disadvantages in the community. As far as I am concerned, I oppose to buid a ne factory in my neighborhood. This is becuase there is more disadvantages of having a factory to local residents than benefits.\r\n\r\nA new factory could be beneficial in a place like ghetto, where many beggars and unemploys live. People in here can get a job and earn enough money so that they could eat and support their family. These days, many people are out of work and depend upon the welfare system. Therefore, a new factory could be a kind of hope for these unemployed people. Also, once a ew big business starts up, other small buisinesses tend to follow. From this community will have a chance to grow and develop.\r\n\r\nOn the other hand, people should consider about the environmental problem factory will bring. Tody, pollution is the biggest problem people have to solve.  People have already 